In [ ]:
import pickle

import numpy
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
# load sample data
path = "/Users/paraspokharel/Programming/pitchProphet/pitchProphet/data/football_data_uk/Bundesliga_16_17.csv"
data = pd.read_csv(path)
data

In [ ]:
# prepare training data

label_map = {"H": 0, "D": 1, "A": 2}
x_df = data.drop(columns=["HomeTeam", "AwayTeam", "Date", "FTR"])
y_df = data["FTR"].map(label_map)
x_train, x_test = train_test_split(x_df, test_size=0.3, random_state=42)
y_train, y_test = train_test_split(y_df, test_size=0.3, random_state=42)

In [ ]:
# train boost

model = xgb.XGBClassifier(
    objective="multi:softmax",
    num_class=3,
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
)

# Fit the model with early stopping and evaluation metric
model.fit(
    x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)], verbose=False
)
# TODO: optimize

In [ ]:
# evaluate
eval_result = model.evals_result()
pred_y = model.predict_proba(x_test)
auc = roc_auc_score(y_test, pred_y, multi_class="ovo")

In [ ]:
# download model (pickle converts to byte stream)
pickle.dump(model, open(r"../xgb_model.pkl", "wb"))